In [46]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader 
from torch.utils.tensorboard import SummaryWriter

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.relu = nn.ReLU()
        self.pool = nn.AvgPool2d(kernel_size=(2, 2), stride=(2, 2))
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=(5, 5), stride = (1, 1), padding = (0, 0))
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=(5, 5), stride = (1, 1), padding = (0, 0))
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=(5, 5), stride = (1, 1), padding = (0, 0))
        self.linear1 = nn.Linear(120, 84)
        self.linear2 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.relu(self.conv3(x))
        x = x.reshape(x.shape[0], -1)
        x = self.relu(self.linear1(x))
        x = self.linear2(x)
        return x



In [47]:
x = torch.randn(64, 1, 32, 32)
model = LeNet()
print(model(x).shape)

torch.Size([64, 10])


In [48]:
device = "cuda" if torch.cuda.is_available else "cpu"

In [49]:
train_transforms = transforms.Compose([
    transforms.Resize(size=(32, 32)),
    transforms.ToTensor()
])

In [50]:
train_dataset = datasets.MNIST(root = "../../data/mnist", train = True, transform=train_transforms, download = True)

In [51]:
model.to(device)
loss_fn = nn.CrossEntropyLoss()
EPOCHS = 5

In [52]:
write = SummaryWriter(
            f"runs/LeNet"
        )

In [53]:
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
optimizer = optim.Adam(model.parameters(), lr = 0.01, weight_decay=0.0)
for epoch in range(EPOCHS):
    losses = []
    accuracies = []
    write = SummaryWriter(
            f"runs/LeNet/{epoch}"
        )
    for batch, (data, label) in enumerate(train_dataloader):
        data = data.to(device)
        label = label.to(device)

        y_pred = model(data)
        loss = loss_fn(y_pred, label)
        losses.append(loss.item())

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()

        _, prediction = y_pred.max(1)
        num_correct = (prediction == label).sum()
        running_acc = float(num_correct)/float(data.shape[0])

        write.add_scalar("Training loss", loss, global_step=batch)
        write.add_scalar(
            "Training Accuracy", running_acc, global_step=batch
        )